# Autoparametric resonance

<center><img src="mass-swing.png" width="15%" height="auto"/></center>

System of equations of motion for the system:
\begin{aligned}
\ddot{R}&=R\dot{\theta}^2-\frac{k}{m}(R-l)+g\cos\theta+a\cos\Omega t\\
\ddot{\theta}&=-\frac{2}{R}\dot{R}\dot{\theta}-\frac{g}{m}\sin\theta
\end{aligned}

Find stationary point for linearization $\theta=0$, all derivatives are zero:
$$-\frac{k}{m}(R-l)+g=0\quad\implies\quad R=l+\frac{mg}{k}$$
linearize equations around stable point
$$R=R_0+r,\quad R_0=l+mg/k,\quad\theta_0=0$$
where $r/R_0\ll1$ and $\theta\ll1$. Scale time to get rid of frequencies $t'=\omega_1t$ assuming $\omega_1=2\omega_2$.
\begin{aligned}
\ddot{r}+\frac{1}{Q_1}\dot{r}+r&=l\dot{\theta}^2+a\cos\Omega t\\
\ddot{\theta}+\frac{1}{Q_2}\dot{\theta}+\frac{1}{4}\theta&=-2\dot{r}\dot{\theta}
\end{aligned}

Following to the averaging method:
\begin{aligned}
r &= u_1\cos\Omega t+v_1\sin\Omega t\\
\theta &= u_2\cos\frac{\Omega}{2}t+v_2\sin\frac{\Omega}{2}t\\
\dot{r} &= \Omega (-u_1\sin\Omega t+v_1\cos\Omega t)\\
\dot{\theta} &= \frac{\Omega}{2} (-u_2\sin\frac{\Omega}{2}t+v_2\cos\frac{\Omega}{2}t)
\end{aligned}
with additional constraints
\begin{aligned}
\dot{u}_1\cos\Omega t+\dot{v}_1\sin\Omega t &= 0\\
\dot{u}_2\cos\frac{\Omega}{2}t+\dot{v}_2\sin\frac{\Omega}{2}t &= 0
\end{aligned}

Introduce detuning parameters
$$\xi_1=1-\Omega^2,\quad\xi_2=1-\Omega^2/4$$

In [1]:
import sympy as sp

In [2]:
t, l, a, Q1, Q2 = sp.symbols('t, l, a, Q_1, Q_2')
u1, u2, v1, v2 = sp.symbols('u_1, u_2, v_1, v_2')
xi1, xi2 = sp.symbols('\\xi_1, \\xi_2')
W = sp.Symbol('\Omega', positive=True)

In [3]:
r  =       u1 * sp.cos(W*t) + v1 * sp.sin(W*t)
dr = W * (-u1 * sp.sin(W*t) + v1 * sp.cos(W*t))

In [4]:
th  =         u2 * sp.cos(W/2*t) + v2 * sp.sin(W/2*t)
dth = W/2 * (-u2 * sp.sin(W/2*t) + v2 * sp.cos(W/2*t))

In [5]:
F1 = -dr/Q1/W  - xi1/W * r + l/W * dth**2 + a/W *sp.cos(W*t)
F2 = -dth/Q2/W - xi2/W * th - 2/l/(W/2) * dr * dth

In [6]:
T = 4*sp.pi/W

In [7]:
du1 = (sp.integrate(-sp.sin(W*t)*F1, (t, 0, T))/T).expand()
dv1 = (sp.integrate( sp.cos(W*t)*F1, (t, 0, T))/T).expand()

In [8]:
du2 = (sp.integrate(-sp.sin(W/2*t)*F2, (t, 0, T))/T).expand()
dv2 = (sp.integrate( sp.cos(W/2*t)*F2, (t, 0, T))/T).expand()

Get equations for the steady-state solution.

In [9]:
eq1=du1*2
eq1

\Omega*l*u_2*v_2/4 + \xi_1*v_1/\Omega - u_1/Q_1

In [10]:
eq2=dv1*2
eq2

-\Omega*l*u_2**2/8 + \Omega*l*v_2**2/8 - \xi_1*u_1/\Omega + a/\Omega - v_1/Q_1

In [11]:
eq3 = du2*2
eq3

-\Omega*u_1*v_2/l + \Omega*u_2*v_1/l + \xi_2*v_2/\Omega - u_2/(2*Q_2)

In [12]:
eq4=dv2*2
eq4

-\Omega*u_1*u_2/l - \Omega*v_1*v_2/l - \xi_2*u_2/\Omega - v_2/(2*Q_2)

Replace real $u$, $v$ with complex analogs to easy account for damping
$$z_1=u_1+iv_1,\quad z_2=u_1-iv_1,\quad z_3=u_2+iv_2,\quad z_4=u_2-iv_2$$

In [13]:
z1, z2, z3, z4=sp.symbols('z_1, z_2, z_3, z_4')

In [14]:
sbs=[(u1, z1/2+z2/2), (v1, -sp.I*z1/2+sp.I*z2/2),
     (u2, z3/2+z4/2), (v2, -sp.I*z3/2+sp.I*z4/2)]

In [15]:
eq5=eq1.subs(sbs).expand()
eq6=eq2.subs(sbs).expand()
eq7=eq3.subs(sbs).expand()
eq8=eq4.subs(sbs).expand()

In [16]:
eq9  = (eq5 + sp.I * eq6).expand()
eq10 = (eq5 - sp.I * eq6).expand()
eq11 = (eq7 + sp.I * eq8).expand()
eq12 = (eq7 - sp.I * eq8).expand()

In [17]:
eq9

-I*\Omega*l*z_3**2/8 - I*\xi_1*z_1/\Omega + I*a/\Omega - z_1/Q_1

In [18]:
eq10

I*\Omega*l*z_4**2/8 + I*\xi_1*z_2/\Omega - I*a/\Omega - z_2/Q_1

In [19]:
eq11

-I*\Omega*z_1*z_4/l - I*\xi_2*z_3/\Omega - z_3/(2*Q_2)

In [20]:
eq12

I*\Omega*z_2*z_3/l + I*\xi_2*z_4/\Omega - z_4/(2*Q_2)

In [21]:
sol1 = sp.solve((eq9,eq10),[z1,z2])
Z1 = sol1[z1].expand()
Z2 = sol1[z2].expand()

In [22]:
Z1

-Q_1*\Omega**2*l*z_3**2/(8*Q_1*\xi_1 - 8*I*\Omega) + 8*Q_1*a/(8*Q_1*\xi_1 - 8*I*\Omega)

In [23]:
Z2

-Q_1*\Omega**2*l*z_4**2/(8*Q_1*\xi_1 + 8*I*\Omega) + 8*Q_1*a/(8*Q_1*\xi_1 + 8*I*\Omega)

Substitute them into the second couple of equations

In [24]:
eq13 = eq11.subs([(z1,Z1),(z2,Z2)]).expand()
eq13

I*Q_1*\Omega**3*z_3**2*z_4/(8*Q_1*\xi_1 - 8*I*\Omega) - 8*I*Q_1*\Omega*a*z_4/(8*Q_1*\xi_1*l - 8*I*\Omega*l) - I*\xi_2*z_3/\Omega - z_3/(2*Q_2)

In [25]:
eq14 = eq12.subs([(z1,Z1),(z2,Z2)]).expand()
eq14

-I*Q_1*\Omega**3*z_3*z_4**2/(8*Q_1*\xi_1 + 8*I*\Omega) + 8*I*Q_1*\Omega*a*z_3/(8*Q_1*\xi_1*l + 8*I*\Omega*l) + I*\xi_2*z_4/\Omega - z_4/(2*Q_2)

Let the amplitudes of modes are
$$A_1^2 = z_1z_2 = u_1^2+v_1^2,\quad A_2^2 = z_3z_4 = u_2^2+v_2^2$$
The trivial solution to the system is $z_3=z_4=0$, $A_2=0$ and
\begin{equation}
z_{1,2}=\frac{a}{\pm\xi_1-\Omega/{iQ_1}},\quad A_1=\frac{a}{\sqrt{(1-\Omega^2)^2+\Omega^2/Q_1^2}}
\end{equation}

The system has nontrivial value

In [26]:
A1, A2 = sp.symbols('A1, A2')

In [27]:
eq15 = Q1*W**3*z3*A2**2/(-8*sp.I*Q1*xi1 - 8*W) - 8*Q1*W*a*z4/(-8*sp.I*Q1*xi1*l - 8*W*l) - sp.I*xi2*z3/W - z3/(2*Q2)
eq16 = Q1*W**3*z4*A2**2/(8*sp.I*Q1*xi1 - 8*W) - 8*Q1*W*a*z3/(8*sp.I*Q1*xi1*l - 8*W*l) + sp.I*xi2*z4/W - z4/(2*Q2)

In [28]:
eq17 = eq15.collect([z3,z4])
eq18 = eq16.collect([z3,z4])

The linear system of two equations for $z_3$ and $z_4$ has a nontrivial solution when determinant is zero what yields $A_2^2$ value. Since A_2 is amplitude, nontrivial solution will be stable when $A_2$ has a real positive value, otherwise the solution is trivial.

In [29]:
eq17

-8*Q_1*\Omega*a*z_4/(-8*I*Q_1*\xi_1*l - 8*\Omega*l) + z_3*(A2**2*Q_1*\Omega**3/(-8*I*Q_1*\xi_1 - 8*\Omega) - I*\xi_2/\Omega - 1/(2*Q_2))

In [30]:
eq18

-8*Q_1*\Omega*a*z_3/(8*I*Q_1*\xi_1*l - 8*\Omega*l) + z_4*(A2**2*Q_1*\Omega**3/(8*I*Q_1*\xi_1 - 8*\Omega) + I*\xi_2/\Omega - 1/(2*Q_2))